In [ ]:
# Reading the data from Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
from os.path import isfile

os.chdir("/content/drive/MyDrive/example-set")
filelist = [filename for filename in os.listdir() if isfile(filename)]

In [ ]:
 # Jay
from collections import defaultdict

class Client:

    def __init__(self, ID, currencies, positionCheck, rating):
        self.ID = ID
        self.currencies = currencies
        self.positionCheck = positionCheck
        self.rating = int(rating)

        self.interimPositions = defaultdict(lambda: 0)
        self.actualPositions = defaultdict(lambda: 0)

    def canShortSell(self, orderObj): #takes in orderObj, returns bool
        # if self.positionCheck == 'N' or orderObj.side == 'Buy':
        #     return True

        # currHoldingQty = self.interimPositions[orderObj.ID][0]
        # if (orderObj.quantity > currHoldingQty):
        #     return False

        return True

    def updateInterimPositions(self, orderObj):
        if orderObj.side == 'Buy':
            self.interimPositions[orderObj.ID] += (orderObj.quantity, orderObj.price)
        else:
            self.interimPositions[orderObj.ID][0] -= orderObj.quantity


    def updateActualPositions(self, orderObj):
        if orderObj.side == 'Buy':
            self.actualPositions[orderObj.ID] += (orderObj.quantity, orderObj.price)
        else:
            self.actualPositions[orderObj.ID][0] -= orderObj.quantity



class Instrument:
    def __init__(self, ID, currency, lotSize):
        self.ID = ID
        self.currency = currency
        self.lotSize = lotSize
        self.history = defaultdict(lambda: 0) #price:qty

    def updateHistory(self, price, quantity):
        self.history[price] += quantity


class Order:
    def __init__(self, timeStamp, ID, instrumentStr, quantity, clientStr, price, side):
        self.time = timeStamp if len(timeStamp) > 7 else '0' + timeStamp
        self.ID = ID
        self.instrument = instrumentStr
        self.quantity = quantity
        self.client = clientStr
        self.price = price
        self.side = side

    def __lt__(self, other):
      return self.side < other.side

    def updateQuantity(self, quantity):
        self.quantity = quantity

    def updatePrice(self, price):
      self.price = price


In [ ]:
# JX
from heapq import heappush, heappop

class OrderBook:
  def __init__(self):
    self.marketOrders = []
    self.limitOrders = []

class BidOrderBook(OrderBook):
  def addMarketOrder(self, order):
    sortBy = (clients[order.client].rating, order.time)
    heappush(self.marketOrders, [sortBy, order])

  def addLimitOrder(self, order):
    # maxHeap for Bid Order Book
    sortBy = (-float(order.price), clients[order.client].rating, order.time)
    heappush(self.limitOrders, [sortBy, order])

class AskOrderBook(OrderBook):
  def addMarketOrder(self, order):
    sortBy = (clients[order.client].rating, order.time)
    heappush(self.marketOrders, [sortBy, order])

  def addLimitOrder(self, order):
    # minHeap for Ask Order Book
    sortBy = (float(order.price), clients[order.client].rating, order.time)
    heappush(self.limitOrders, [sortBy, order])

class MatchingEngine:
  def __init__(self):
    self.bidOrderBook = BidOrderBook()
    self.askOrderBook = AskOrderBook()

    self.state = 'OPEN_AUCTION'

  def addOrder(self, order):
    if order.time > '09:30:00':
      self.state = 'CONTINUOUS_TRADING'
    elif order.time > '16:00:00':
      self.state = 'CLOSE_AUCTION'

    if order.side == 'Buy' and order.price == 'Market':
      self.bidOrderBook.addMarketOrder(order)
    elif order.side == 'Buy':
      self.bidOrderBook.addLimitOrder(order)
    elif order.side == 'Sell' and order.price == 'Market':
      self.askOrderBook.addMarketOrder(order)
    elif order.side == 'Sell':
      self.askOrderBook.addLimitOrder(order)

    if self.state == 'OPEN_AUCTION':
        pass

  def checkOrder(self, order):
    if not clients[order.client].canShortSell(order):
      rejectedOrders.append([order, "REJECTED-POSITION CHECK FAILED"])
      return

    if not instruments_data_specification(order, instrument):
      rejectedOrders.append([order, "REJECTED-INVALID LOT SIZE"])
      return

    currencies = clients[order.client].currencies.split(',')
    currency = instruments[order.instrument].currency

    if currency not in currencies:
      rejectedOrders.append([order, "REJECTED-MISMATCH CURRENCY"])
      return

    if order.instrument not in instruments:
      rejectedOrders.append([order, "REJECTED-INSTRUMENT NOT FOUND"])
      return

    self.addOrder(order)

    return rejectedOrders

  def resolveOpenAuction(self):
    bidVol = defaultdict(int)
    bidMarket = 0

    askVol = defaultdict(int)
    askMarket = 0

    prices = set()

    for sortBy, order in self.bidOrderBook.marketOrders:
      bidMarket += int(order.quantity)

    for sortBy, order in self.askOrderBook.marketOrders:
      askMarket += int(order.quantity)

    for sortBy, order in self.bidOrderBook.limitOrders:
      prices.add(float(order.price))
      bidVol[float(order.price)] += int(order.quantity)

    for sortBy, order in self.askOrderBook.limitOrders:
      prices.add(float(order.price))
      askVol[float(order.price)] += int(order.quantity)

    # print(prices)
    # print("Bid:")
    # print(bidVol)
    # print(bidMarket)
    # print("Ask:")
    # print(askVol)
    # print(askMarket)

    finalQuantity, finalPrice = 0, 0

    for matchPrice in prices:
      buyQuantity, sellQuantity = bidMarket, askMarket

      for price, quantity in bidVol.items():
        if price >= matchPrice: buyQuantity += quantity

      for price, quantity in askVol.items():
        if price <= matchPrice: sellQuantity += quantity

      if min(buyQuantity, sellQuantity) > finalQuantity:
        finalQuantity, finalPrice = min(buyQuantity, sellQuantity), matchPrice

    # print(finalQuantity, finalPrice)

    buyQuantity, sellQuantity = finalQuantity, finalQuantity
    # check Bid Order Book Market Order
    while self.bidOrderBook.marketOrders:
      if buyQuantity <= 0: break

      #if bidOrderBook.marketOrders[0][1].quantity


  def matchContinuousOrder(self, order, side, orderType):
        fulfilledOrders = []

        if (orderType == 'Market'):
            if (side == 'Buy' and self.askOrderBook.limitOrders):
                ask = self.askOrderBook.limitOrders[0][1]
                if (order.quantity > ask.quantity):
                    order.updateQuantity(order.quantity-ask.quantity)
                    fulfilledOrders.append(heappop(self.askOrderBook.limitOrders))

                elif (order.quantity < ask.quantity):
                    ask.updateQuantity(ask.quantity - order.quantity)
                    order.updatePrice(ask.price)
                    fulfilledOrders.append(order)
                    self.bidOrderBook.marketOrders = [x for x in self.bidOrderBook.marketOrders if x[1] != order]

                else:
                    fulfilledOrders.append(order)
                    fulfilledOrders.append(ask)
                    heappop(self.askOrderBook.limitOrders)
                    self.bidOrderBook.marketOrders = [x for x in self.bidOrderBook.marketOrders if x[1] != order]


            elif (side == 'Sell' and self.bidOrderBook.limitOrders):
                bid = self.bidOrderBook.limitOrders[0][1]
                if (order.quantity > bid.quantity):
                    order.updateQuantity(order.quantity-bid.quantity)
                    fulfilledOrders.append(heappop(self.bidOrderBook.limitOrders))

                elif (order.quantity < bid.quantity):
                    bid.updateQuantity(bid.quantity - order.quantity)
                    order.updatePrice(bid.price)
                    fulfilledOrders.append(order)
                    self.askOrderBook.marketOrders = [x for x in self.askOrderBook.marketOrders if x[1] != order]

                else:
                    order.updatePrice(bid.price)
                    fulfilledOrders.append(order)
                    fulfilledOrders.append(bid)
                    heappop(self.bidOrderBook.limitOrders)
                    print(self.askOrderBook.marketOrders)
                    self.askOrderBook.marketOrders = [x for x in self.askOrderBook.marketOrders if x[1] != order]

        elif (orderType == 'Limit'):
            if (side == 'Buy' and self.askOrderBook.marketOrders):
                ask = self.askOrderBook.marketOrders[0][1]
                if (order.quantity > ask.quantity):
                    order.updateQuantity(order.quantity-ask.quantity)
                    fulfilledOrders.append(heappop(self.askOrderBook.marketOrders))

                elif (order.quantity < ask.quantity):
                    ask.updateQuantity(ask.quantity - order.quantity)
                    order.updatePrice(ask.price)
                    fulfilledOrders.append(order)
                    self.bidOrderBook.limitOrders.remove(order)

                else:
                    order.updatePrice(ask.price)
                    fulfilledOrders.append(order)
                    fulfilledOrders.append(ask)
                    self.bidOrderBook.limitOrders = [x for x in self.bidOrderBook.limitOrders if x[1] != order]
                    heappop(self.askOrderBook.marketOrders)

            elif (side == 'Sell' and self.bidOrderBook.marketOrders):
                bid = self.bidOrderBook.marketOrders[0][1]
                if (order.quantity > bid.quantity):
                    order.updateQuantity(order.quantity-bid.quantity)
                    fulfilledOrders.append(self.bidOrderBook.marketOrders.heappop)

                elif (order.quantity < bid.quantity):
                    bid.updateQuantity(bid.quantity - order.quantity)
                    order.updatePrice(bid.price)
                    fulfilledOrders.append(order)
                    self.askOrderBook.limitOrders = [x for x in self.askOrderBook.limitOrders if x[1] != order]

                else:
                    order.updatePrice(bid.price)
                    fulfilledOrders.append(order)
                    fulfilledOrders.append(bid)
                    heappop(self.bidOrderBook.marketOrders)
                    self.askOrderBook.limitOrders = [x for x in self.askOrderBook.limitOrders if x[1] != order]

            else: # limit-limit order
                if (orderType == 'Buy' and self.bidOrderBook.limitOrders[0][1] == order):
                    ask = self.askOrderBook.limitOrders[0][1]
                    if order.price >= ask.price:
                        if (order.quantity > ask.quantity):
                            fulfilledOrders.append(self.askOrderBook.limitOrders.heappop)
                            totalOrderQty = order.quantity - ask.quantity
                            while totalOrderQty <= order.quantity:
                                curr = self.askOrderBook.limitOrders[0][1]

                                if totalOrderQty + curr.quantity == order.quantity:
                                    fulfilledOrders.append(order)
                                    fulfilledOrders.append(self.askOrderBook.limitOrders.heappop)
                                    break

                                if totalOrderQty + curr.quantity > order.quantity:
                                    break

                                totalOrderQty += curr.quantity
                                fulfilledOrders.append(self.askOrderBook.limitOrders.heappop)
        print("FULFILLED",fulfilledOrders)
        if fulfilledOrders:
            print("FULFILLED",fulfilledOrders)
            for successOrder in fulfilledOrders:
                self.fulfilOrder(successOrder)

  def fulfilOrder(self, orderObj):
          global clients, instruments

          clients[orderObj.client].updateActualPositions(orderObj)
          instruments[orderObj.instrument].updateHistory(orderObj.price, orderObj.quantity)

In [ ]:
# Read csv
from collections import defaultdict
import csv

def read_csv(filename):
    with open(filename,'r') as csvfile:
        file = csv.DictReader(csvfile)
        return list(file)  # Read the contents of the file into a list

def clients_init(rows):
    clients = defaultdict(lambda: 0)
    for row in rows:
      obj = Client(row['ClientID'], row['Currencies'], row['PositionCheck'], row['Rating'])
      clients[obj.ID] = obj
    return clients

def instruments_init(rows):
    instruments = defaultdict(lambda: 0)
    for row in rows:
      obj = Instrument(row['InstrumentID'], row['Currency'], row['LotSize'])
      instruments[obj.ID] = obj
    return instruments

def orders_init(rows):
    orders = defaultdict(lambda: 0)
    for row in rows:
      obj = Order(row['Time'], row['OrderID'], row['Instrument'], row['Quantity'], row['Client'], row['Price'], row['Side'])
      orders[obj.ID] = obj
    return orders

# Assuming filelist is a list of filenames
for filename in filelist:
    file = read_csv(filename)
    if filename == 'input_clients.csv':
        clients = clients_init(file)
    elif filename == 'input_instruments.csv':
        instruments = instruments_init(file)
    elif filename == 'input_orders.csv':
        orders = orders_init(file)

In [ ]:
print(clients)
print(instruments)
print(orders)

defaultdict(<function clients_init.<locals>.<lambda> at 0x7f8d959e3c70>, {'A': <__main__.Client object at 0x7f8db813bf10>, 'B': <__main__.Client object at 0x7f8db8138fa0>, 'C': <__main__.Client object at 0x7f8db813b370>, 'D': <__main__.Client object at 0x7f8db8139ed0>, 'E': <__main__.Client object at 0x7f8db81390c0>})
defaultdict(<function instruments_init.<locals>.<lambda> at 0x7f8d9596ac20>, {'SIA': <__main__.Instrument object at 0x7f8db813b100>})
defaultdict(<function orders_init.<locals>.<lambda> at 0x7f8d95968c10>, {'A1': <__main__.Order object at 0x7f8db813bf40>, 'B1': <__main__.Order object at 0x7f8db813b2b0>, 'C1': <__main__.Order object at 0x7f8db8139a20>, 'D1': <__main__.Order object at 0x7f8db81386d0>, 'B2': <__main__.Order object at 0x7f8db813bc10>, 'E1': <__main__.Order object at 0x7f8db813b070>, 'A2': <__main__.Order object at 0x7f8db813ae90>, 'C2': <__main__.Order object at 0x7f8db8139ab0>, 'B3': <__main__.Order object at 0x7f8db8139b40>, 'C3': <__main__.Order object at 

In [ ]:
import math

def instruments_data_specification(Order, Instrument):
  if (math.trunc(float(Order.quantity)) % math.trunc(float(Instrument.lotSize)) != 0):
    return False

  return True

In [ ]:
print(orders)

defaultdict(<function orders_init.<locals>.<lambda> at 0x7f8d95968c10>, {'A1': <__main__.Order object at 0x7f8db813bf40>, 'B1': <__main__.Order object at 0x7f8db813b2b0>, 'C1': <__main__.Order object at 0x7f8db8139a20>, 'D1': <__main__.Order object at 0x7f8db81386d0>, 'B2': <__main__.Order object at 0x7f8db813bc10>, 'E1': <__main__.Order object at 0x7f8db813b070>, 'A2': <__main__.Order object at 0x7f8db813ae90>, 'C2': <__main__.Order object at 0x7f8db8139ab0>, 'B3': <__main__.Order object at 0x7f8db8139b40>, 'C3': <__main__.Order object at 0x7f8db813ba90>, 'B4': <__main__.Order object at 0x7f8db8139e70>, 'E2': <__main__.Order object at 0x7f8db813b8e0>, 'B5': <__main__.Order object at 0x7f8db813bbb0>, 'C4': <__main__.Order object at 0x7f8db8138370>, 'B6': <__main__.Order object at 0x7f8db813b400>, 'A3': <__main__.Order object at 0x7f8db813bb80>, 'E3': <__main__.Order object at 0x7f8db813bf70>})


In [ ]:
matchingEngine = MatchingEngine()
rejectedOrders = []

for order in orders:
  client = clients[order]
  instrument = instruments[orders[order].instrument]
  print(order)
  print(instrument)
  print(client)
  #matchingEngine.checkOrder(order,instrument,client,rejectedOrders)

A1
0
B1
0
C1
0
D1
0
B2
0
E1
0
A2
0
C2
0
B3
0
C3
0
B4
0
E2
0
B5
0
C4
0
B6
0
A3
0
E3
0


In [ ]:
# Test
matchingEngine = MatchingEngine()

for order in orders.values():
 matchingEngine.checkOrder(order)

# for client in clients.values():
#  print(client.actualPositions)

for instrument in instruments.values():
 print(instrument.history)

defaultdict(<function Instrument.__init__.<locals>.<lambda> at 0x7f8d95969bd0>, {})


In [ ]:
import unittest

class TestOrderBook(unittest.TestCase):
  matchingEngine = MatchingEngine()
  rejectedOrders = []

  for order in orders.values():
    matchingEngine.checkOrder(order)

  matchingEngine.resolveOpenAuction()

  # while matchingEngine.askOrderBook.marketOrders:
  #   print(heappop(matchingEngine.askOrderBook.marketOrders))

  # while matchingEngine.askOrderBook.limitOrders:
  #   print(heappop(matchingEngine.askOrderBook.limitOrders))

  # while matchingEngine.bidOrderBook.marketOrders:
  #   print(heappop(matchingEngine.bidOrderBook.marketOrders))

  # while matchingEngine.bidOrderBook.limitOrders:
  #   print(heappop(matchingEngine.bidOrderBook.limitOrders))

  # for reject in rejectedOrders:
  #   print(reject)

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
def outputExchangeReport(rejectedOrders):
  filename = 'output_exchange_report.csv'
  with open(filename, 'w',newline='') as csvfile:
    fields = ['Order Id', 'Rejection Reason']
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    for order,reason in rejectedOrders:
      writer.writerow({'Order Id': order.ID, 'Rejection Reason': reason})

outputExchangeReport(rejectedOrders)